In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv("LANGCHAIN_ENDPOINT")  
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['OPENAI_API_KEY']= os.getenv("OPENAI_API_KEY")

In [8]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini",
                temperature=0.7, # Controls randomness: higher values (e.g., 1.0) for more creative, lower (e.g., 0.2) for more deterministic
                top_p=1.0, # Nucleus sampling: controls diversity by sampling from the most probable tokens
                frequency_penalty=0.0, # Penalizes new tokens based on their existing frequency in the text so far
                presence_penalty=0.0, # Penalizes new tokens based on whether they appear
                logit_bias={}, # Modifies the likelihood of specific tokens appearing in the output\
                response_format={"type": "text"}, # Specifies the format of the output (e.g., "text", "json_object")
                seed=None, # For reproducible outputs
                tools=[], # List of tools the model can call
                tool_choice="auto", # Controls how the model uses tools
                logprobs=False, # Whether to return log probabilities of output tokens
                top_logprobs=None # Number of top log probabilities to return
                 )
# print(llm.invoke("Who is man's best friend?"))

c:\Users\002PMB744\Desktop\RAG-Langchain\Env-RAG\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\002PMB744\Desktop\RAG-Langchain\Env-RAG\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! tools is not default parameter.
                tools was transferred to model_kwargs.
                Please confirm that tools is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\002PMB744\Desktop\RAG-Langchain\Env-RAG\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! tool_choice is not default parameter.
                tool_choice was transferred to model_kwargs.
                Please confirm that tool_choice is what you intended.
  if await 

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


AIMessage(content='The capital of France is Paris.', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 30, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run-9eacf883-07c7-4a10-8c2a-724084363d67-0', usage_metadata={'input_tokens': 30, 'output_tokens': 8, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
